## Hertz Contact
#### Contact between a sphere and a half-space

An elastic sphere of radius $R$ indents an elastic half-space where total deformation is $d$, causing a contact area of radius[1]

![Alt Text](/Users/lizichen/Master_internship/Week2/Contact_sphere-plane.jpg)

$$
a=\sqrt{R d}
$$

The applied force $F$ is related to the displacement $d$ by

$$
F=\frac{4}{3} E^* R^{\frac{1}{2}} d^{\frac{3}{2}}
$$

where 

$$
\frac{1}{E^*}=\frac{1-\nu_1^2}{E_1}+\frac{1-\nu_2^2}{E_2}
$$

and $E_1, E_2$ are the elastic moduli and $\nu_1, \nu_2$ the Poisson's ratios associated with each body.

The distribution of normal pressure in the contact area as a function of distance from the center of the circle is 

$$
p(r)=p_0\left(1-\frac{r^2}{a^2}\right)^{\frac{1}{2}}
$$

where $p_0$ is the maximum contact pressure given by

$$
p_0=\frac{3 F}{2 \pi a^2}=\frac{1}{\pi}\left(\frac{6 F E^{* 2}}{R^2}\right)^{\frac{1}{3}}
$$

The radius of the circle is related to the applied load $F$ by the equation


In [1]:
import numpy as np
import matplotlib as plt
import time

##### Reference:

[1] https://en.wikipedia.org/wiki/Contact_mechanics